# **Automation of the CLI commands of the Snakemake workflow**

This Jupyter Notebook has the objective of automating Snakemake commands.

While it can be useful for quick tasks, using this instead of the command line may not provide you with the ability to access some of Snakemake's advanced features.

* Environment to use: [Greference_tools](code/environments/Greference_tools.yml)

In [1]:
## Libreries
import subprocess
import os
import re

In [2]:
## lists for the loops
chr_list        = ["chr3", "chr5", "chr7", "chr12", "chr17"]

sample_list     = ["ERR696683", "ERR753368", "ERR753369" ,"ERR753370" ,"ERR753371" ,"ERR753372", 
                  "ERR753373", "ERR753374", "ERR753375", "ERR753376", "ERR753377", "ERR753378"]


1. **rule download_data**

In [ ]:
subprocess.run(["snakemake", "--cores", "1", "download_data"])

* rule pre_processing

Filtering the chromosomes, make sure that the chromosomes of the config file are **ch3**. 

In [3]:
## Running the 3º chromosome
subprocess.run(["snakemake", "--cores", "1", "data/original_bam/filtering/ERR696683_chr3_sorted.bam"])

## Running the rest of the chromosomes
for chr in range(len(chr_list) - 1):
    chr1 = chr_list[chr]
    chr2 = chr_list[chr + 1]
    
    print("""
          Chromosomes transformed:
          """, chr1, chr2)
    
    subprocess.run(["sed", "-i", "6,+12s/" + chr1 + "/" + chr2 + "/g", "config.yaml"])
    subprocess.run(["sed", "-i", "39s/" + chr1 + "/" + chr2 + "/g", "config.yaml"])

    subprocess.run(["snakemake", "--cores", "1", "data/original_bam/filtering/ERR696683_" + chr2 + "_sorted.bam"])

Building DAG of jobs...
Using shell: /usr/bin/bash
Provided cores: 1 (use --cores to define parallelism)
Rules claiming more threads will be scaled down.
Conda environments: ignored
Job stats:
job               count    min threads    max threads
--------------  -------  -------------  -------------
pre_processing        1              1              1
total                 1              1              1

Select jobs to execute...

[Mon Sep 25 16:01:21 2023]
rule pre_processing:
    input: code/03extracting_fastq.sh
    output: data/original_bam/filtering/ERR696683_chr3_sorted.bam
    jobid: 0
    reason: Missing output files: data/original_bam/filtering/ERR696683_chr3_sorted.bam
    wildcards: sample=ERR696683_chr3
    resources: tmpdir=/tmp

[bam_sort_core] merging from 1 files and 1 in-memory blocks...
[M::bam2fq_mainloop] discarded 84123 singletons
[M::bam2fq_mainloop] processed 4225255 reads
[bam_sort_core] merging from 1 files and 1 in-memory blocks...
[M::bam2fq_mainloop] disca


          Chromosomes transformed:
           chr3 chr5


Building DAG of jobs...
Using shell: /usr/bin/bash
Provided cores: 1 (use --cores to define parallelism)
Rules claiming more threads will be scaled down.
Conda environments: ignored
Job stats:
job               count    min threads    max threads
--------------  -------  -------------  -------------
pre_processing        1              1              1
total                 1              1              1

Select jobs to execute...

[Mon Sep 25 16:43:21 2023]
rule pre_processing:
    input: code/03extracting_fastq.sh
    output: data/original_bam/filtering/ERR696683_chr5_sorted.bam
    jobid: 0
    reason: Missing output files: data/original_bam/filtering/ERR696683_chr5_sorted.bam
    wildcards: sample=ERR696683_chr5
    resources: tmpdir=/tmp

[bam_sort_core] merging from 1 files and 1 in-memory blocks...
[M::bam2fq_mainloop] discarded 74911 singletons
[M::bam2fq_mainloop] processed 3565319 reads
[bam_sort_core] merging from 1 files and 1 in-memory blocks...
[M::bam2fq_mainloop] disca


          Chromosomes transformed:
           chr5 chr7


Building DAG of jobs...
Using shell: /usr/bin/bash
Provided cores: 1 (use --cores to define parallelism)
Rules claiming more threads will be scaled down.
Conda environments: ignored
Job stats:
job               count    min threads    max threads
--------------  -------  -------------  -------------
pre_processing        1              1              1
total                 1              1              1

Select jobs to execute...

[Mon Sep 25 17:21:04 2023]
rule pre_processing:
    input: code/03extracting_fastq.sh
    output: data/original_bam/filtering/ERR696683_chr7_sorted.bam
    jobid: 0
    reason: Missing output files: data/original_bam/filtering/ERR696683_chr7_sorted.bam
    wildcards: sample=ERR696683_chr7
    resources: tmpdir=/tmp

[bam_sort_core] merging from 1 files and 1 in-memory blocks...
[M::bam2fq_mainloop] discarded 73902 singletons
[M::bam2fq_mainloop] processed 3944452 reads
[bam_sort_core] merging from 1 files and 1 in-memory blocks...
[M::bam2fq_mainloop] disca


          Chromosomes transformed:
           chr7 chr12


Building DAG of jobs...
Using shell: /usr/bin/bash
Provided cores: 1 (use --cores to define parallelism)
Rules claiming more threads will be scaled down.
Conda environments: ignored
Job stats:
job               count    min threads    max threads
--------------  -------  -------------  -------------
pre_processing        1              1              1
total                 1              1              1

Select jobs to execute...

[Mon Sep 25 18:00:04 2023]
rule pre_processing:
    input: code/03extracting_fastq.sh
    output: data/original_bam/filtering/ERR696683_chr12_sorted.bam
    jobid: 0
    reason: Missing output files: data/original_bam/filtering/ERR696683_chr12_sorted.bam
    wildcards: sample=ERR696683_chr12
    resources: tmpdir=/tmp

[bam_sort_core] merging from 1 files and 1 in-memory blocks...
[M::bam2fq_mainloop] discarded 64449 singletons
[M::bam2fq_mainloop] processed 3693551 reads
[bam_sort_core] merging from 1 files and 1 in-memory blocks...
[M::bam2fq_mainloop] di


          Chromosomes transformed:
           chr12 chr17


Building DAG of jobs...
Using shell: /usr/bin/bash
Provided cores: 1 (use --cores to define parallelism)
Rules claiming more threads will be scaled down.
Conda environments: ignored
Job stats:
job               count    min threads    max threads
--------------  -------  -------------  -------------
pre_processing        1              1              1
total                 1              1              1

Select jobs to execute...

[Mon Sep 25 18:38:31 2023]
rule pre_processing:
    input: code/03extracting_fastq.sh
    output: data/original_bam/filtering/ERR696683_chr17_sorted.bam
    jobid: 0
    reason: Missing output files: data/original_bam/filtering/ERR696683_chr17_sorted.bam
    wildcards: sample=ERR696683_chr17
    resources: tmpdir=/tmp

[bam_sort_core] merging from 1 files and 1 in-memory blocks...
[M::bam2fq_mainloop] discarded 49649 singletons
[M::bam2fq_mainloop] processed 3373755 reads
[bam_sort_core] merging from 1 files and 1 in-memory blocks...
[M::bam2fq_mainloop] di

In [4]:
## reset config 
subprocess.run(["sed", "-i", "6,+12s/chr17/chr3/g", "config.yaml"])
subprocess.run(["sed", "-i", "39s/chr17/chr3/g", "config.yaml"])

CompletedProcess(args=['sed', '-i', '39s/chr17/chr3/g', 'config.yaml'], returncode=0)

## Function for the next rules:

* reference_genome 
* fastqc 
* fastp
* fastqc_trimmed
* bwa_mapping 
* sam_to_bam 
* delete_duplicates 
* extracting_variants 
* vep_install 
* vep_cli 
* parsing_dataR

In [5]:
def snake_workflow(chr: str):
    ## Running the workflow for x chormosome                                                                       
    if chr == "chr3":
        subprocess.run(["snakemake", "--cores", "1", "data/reference/genome.fa"])
    elif chr == "chr5" or chr == "chr7" or chr == "chr12" or chr == "chr17":
        subprocess.run(["snakemake", "--cores", "1", "--force", "data/reference/genome.fa"]) 
    
    for sample in sample_list:
        ## Quality inspection
        subprocess.run([
            "snakemake", "--cores", "1", "--use-conda",
            "results/fastqc_result/" + sample + "_" + chr + "_1_fastqc.html",   
            "data/processed/" + sample + "_" + chr + "_1_fastp.fastq.gz",   
            "results/fastqc_result/trimmed/" + sample + "_" + chr + "_1_fastp_fastqc.html"
        ])
        
    for sample in sample_list:
        ## Mapping reads
        subprocess.run([
            "snakemake", "--cores", "1", "--use-conda",
            "results/mapped_reads/" + sample + "_" + chr + "_sorted.sam",   
            "results/mapped_reads/bam_files/" + sample + "_" + chr + "_sorted.bam",   
            "results/mapped_reads/bam_files/" + sample + "_" + chr + "_dedup.bam"
        ])
        
    for sample in sample_list:
        # Extracting variants  
        subprocess.run([
            "snakemake", "--cores", "1", "--use-conda", 
            "results/variants/" + sample + "_" + chr + ".vcf"
        ])   
    
    ## Downloading VEP 
    if chr == "chr3": 
        subprocess.run(["snakemake", "--cores", "1", "--use-conda", "vep_install_db"])

    ## VEP CLI
    for sample in sample_list:
        subprocess.run([
            "snakemake", "--cores", "1", "--use-conda", 
            "results/variants/vep/" + sample + "_" + chr + ".txt"
        ])
    
    ## Extracting the gene from the VEP files
    subprocess.run(["snakemake", "--cores", "1", "--use-conda", "parsing_dataR"])


## Chromosome 3

In [6]:
snake_workflow("chr3")

Building DAG of jobs...
Using shell: /usr/bin/bash
Provided cores: 1 (use --cores to define parallelism)
Rules claiming more threads will be scaled down.
Conda environments: ignored
Job stats:
job                 count    min threads    max threads
----------------  -------  -------------  -------------
reference_genome        1              1              1
total                   1              1              1

Select jobs to execute...

[Mon Sep 25 19:24:49 2023]
rule reference_genome:
    output: data/reference/genome.fa
    jobid: 0
    reason: Missing output files: data/reference/genome.fa
    resources: tmpdir=/tmp

rm: no se puede borrar 'data/reference/genome.fa*': No existe el archivo o el directorio
--2023-09-25 19:24:49--  https://ftp.ensembl.org/pub/release-109/fasta/homo_sapiens/dna/Homo_sapiens.GRCh38.dna.chromosome.3.fa.gz
Resolviendo ftp.ensembl.org (ftp.ensembl.org)... 

==> If you hadn't a previous reference in the directory and there is an ERROR, is NORMAL <==



193.62.193.169
Conectando con ftp.ensembl.org (ftp.ensembl.org)[193.62.193.169]:443... conectado.
Petición HTTP enviada, esperando respuesta... 200 OK
Longitud: 59761837 (57M) [application/x-gzip]
Guardando como: ‘data/reference/genome.fa.gz’

     0K .......... .......... .......... .......... ..........  0%  463K 2m6s
    50K .......... .......... .......... .......... ..........  0% 5,86M 68s
   100K .......... .......... .......... .......... ..........  0% 1016K 64s
   150K .......... .......... .......... .......... ..........  0%  905K 64s
   200K .......... .......... .......... .......... ..........  0%  912K 64s
   250K .......... .......... .......... .......... ..........  0%  897K 64s
   300K .......... .......... .......... .......... ..........  0% 8,68M 56s
   350K .......... .......... .......... .......... ..........  0%  918K 57s
   400K .......... .......... .......... .......... ..........  0%  948K 57s
   450K .......... .......... .......... .......... ..........

application/gzip


Started analysis of ERR696683_chr3_1.fastq.gz
Approx 5% complete for ERR696683_chr3_1.fastq.gz
Approx 10% complete for ERR696683_chr3_1.fastq.gz
Approx 15% complete for ERR696683_chr3_1.fastq.gz
Approx 20% complete for ERR696683_chr3_1.fastq.gz
Approx 25% complete for ERR696683_chr3_1.fastq.gz
Approx 30% complete for ERR696683_chr3_1.fastq.gz
Approx 35% complete for ERR696683_chr3_1.fastq.gz
Approx 40% complete for ERR696683_chr3_1.fastq.gz
Approx 45% complete for ERR696683_chr3_1.fastq.gz
Approx 50% complete for ERR696683_chr3_1.fastq.gz
Approx 55% complete for ERR696683_chr3_1.fastq.gz
Approx 60% complete for ERR696683_chr3_1.fastq.gz
Approx 65% complete for ERR696683_chr3_1.fastq.gz
Approx 70% complete for ERR696683_chr3_1.fastq.gz
Approx 75% complete for ERR696683_chr3_1.fastq.gz
Approx 80% complete for ERR696683_chr3_1.fastq.gz
Approx 85% complete for ERR696683_chr3_1.fastq.gz
Approx 90% complete for ERR696683_chr3_1.fastq.gz
Approx 95% complete for ERR696683_chr3_1.fastq.gz


...

Started analysis of ERR753377_chr3_1_fastp.fastq.gz
Approx 5% complete for ERR753377_chr3_1_fastp.fastq.gz
Approx 10% complete for ERR753377_chr3_1_fastp.fastq.gz
Approx 15% complete for ERR753377_chr3_1_fastp.fastq.gz
Approx 20% complete for ERR753377_chr3_1_fastp.fastq.gz
Approx 25% complete for ERR753377_chr3_1_fastp.fastq.gz
Approx 30% complete for ERR753377_chr3_1_fastp.fastq.gz
Approx 35% complete for ERR753377_chr3_1_fastp.fastq.gz
Approx 40% complete for ERR753377_chr3_1_fastp.fastq.gz
Approx 45% complete for ERR753377_chr3_1_fastp.fastq.gz
Approx 50% complete for ERR753377_chr3_1_fastp.fastq.gz
Approx 55% complete for ERR753377_chr3_1_fastp.fastq.gz
Approx 60% complete for ERR753377_chr3_1_fastp.fastq.gz
Approx 65% complete for ERR753377_chr3_1_fastp.fastq.gz
Approx 70% complete for ERR753377_chr3_1_fastp.fastq.gz
Approx 75% complete for ERR753377_chr3_1_fastp.fastq.gz
Approx 80% complete for ERR753377_chr3_1_fastp.fastq.gz
Approx 85% complete for ERR753377_chr3_1_fastp.fastq.

Analysis complete for ERR753377_chr3_1_fastp.fastq.gz
application/gzip


Started analysis of ERR753377_chr3_2_fastp.fastq.gz
Approx 5% complete for ERR753377_chr3_2_fastp.fastq.gz
Approx 10% complete for ERR753377_chr3_2_fastp.fastq.gz
Approx 15% complete for ERR753377_chr3_2_fastp.fastq.gz
Approx 20% complete for ERR753377_chr3_2_fastp.fastq.gz
Approx 25% complete for ERR753377_chr3_2_fastp.fastq.gz
Approx 30% complete for ERR753377_chr3_2_fastp.fastq.gz
Approx 35% complete for ERR753377_chr3_2_fastp.fastq.gz
Approx 40% complete for ERR753377_chr3_2_fastp.fastq.gz
Approx 45% complete for ERR753377_chr3_2_fastp.fastq.gz
Approx 50% complete for ERR753377_chr3_2_fastp.fastq.gz
Approx 55% complete for ERR753377_chr3_2_fastp.fastq.gz
Approx 60% complete for ERR753377_chr3_2_fastp.fastq.gz
Approx 65% complete for ERR753377_chr3_2_fastp.fastq.gz
Approx 70% complete for ERR753377_chr3_2_fastp.fastq.gz
Approx 75% complete for ERR753377_chr3_2_fastp.fastq.gz
Approx 80% complete for ERR753377_chr3_2_fastp.fastq.gz
Approx 85% complete for ERR753377_chr3_2_fastp.fastq.

Analysis complete for ERR753377_chr3_2_fastp.fastq.gz


[Mon Sep 25 19:33:43 2023]
Finished job 1.
3 of 3 steps (100%) done
Complete log: .snakemake/log/2023-09-25T193312.527794.snakemake.log
Building DAG of jobs...
Using shell: /usr/bin/bash
Provided cores: 1 (use --cores to define parallelism)
Rules claiming more threads will be scaled down.
Job stats:
job               count    min threads    max threads
--------------  -------  -------------  -------------
fastp                 1              1              1
fastqc                1              1              1
fastqc_trimmed        1              1              1
total                 3              1              1

Select jobs to execute...

[Mon Sep 25 19:33:44 2023]
rule fastp:
    input: data/raw/ERR753378_chr3_1.fastq.gz, data/raw/ERR753378_chr3_2.fastq.gz
    output: data/processed/ERR753378_chr3_1_fastp.fastq.gz, data/processed/ERR753378_chr3_2_fastp.fastq.gz
    jobid: 0
    reason: Missing output files: data/processed/ERR753378_chr3_1_fastp.fastq.gz, data/processed/ERR753378

## Changing the config file to do the chromosome 5

In [ ]:
changes_chr3_to_5 = ["6,+12s/chr3/chr5/g", "24,+12s/chr3/chr5/g", "39s/chr3/chr5/g",
                     "46s/chromosome.3.fa.gz/chromosome.5.fa.gz/g", "65s/PIK3CA/APC/g"]
for change in changes_chr3_to_5:
    subprocess.run(["sed", "-i", change, "config.yaml"])

In [ ]:
snake_workflow("chr5")

## Changing the config file to do the chromosome 7 

In [ ]:
changes_chr5_to_7 = ["6,+12s/chr5/chr7/g", "24,+12s/chr5/chr7/g", "39s/chr5/chr7/g",
                     "46s/chromosome.5.fa.gz/chromosome.7.fa.gz/g", "65s/APC/BRAF/g"]
for change in changes_chr5_to_7:
    subprocess.run(["sed", "-i", change, "config.yaml"])   

In [ ]:
snake_workflow("chr7")

## Changing the config file to do the chromosome 12 

In [ ]:
changes_chr7_to_12 = ["6,+12s/chr7/chr12/g", "24,+12s/chr7/chr12/g", "39s/chr7/chr12/g",
                      "46s/chromosome.7.fa.gz/chromosome.12.fa.gz/g", "65s/BRAF/KRAS/g"]
for change in changes_chr7_to_12:
    subprocess.run(["sed", "-i", change, "config.yaml"])     

In [ ]:
snake_workflow("chr12")

## Changing the config file to do the chromosome 17 

In [ ]:
changes_chr12_to_17 = ["6,+12s/chr12/chr17/g", "24,+12s/chr12/chr17/g", "39s/chr12/chr17/g",
                       "46s/chromosome.12.fa.gz/chromosome.17.fa.gz/g", "65s/KRAS/TP53/g"]
for change in changes_chr12_to_17:
    subprocess.run(["sed", "-i", change, "config.yaml"])

In [ ]:
snake_workflow("chr17")

In [ ]:
## reset config 
changes_chr17_to_3 = ["6,+12s/chr17/chr3/g", "24,+12s/chr17/chr3/g", "39s/chr17/chr3/g",
                      "46s/chromosome.17.fa.gz/chromosome.3.fa.gz/g", "65s/TP53/PIK3CA/g"]
for change in changes_chr17_to_3:
    subprocess.run(["sed", "-i", change, "config.yaml"])

## **Last rule**

### **rule R_plotting**

Plotting the data of the 5 gene tables

In [ ]:
subprocess.run(["snakemake", "--cores", "1", "--use-conda", "R_plotting"])